## Cython orderbook routines 

In [1]:
import alquant.clickdb as clickdb
import cython
from datetime import date
%load_ext cython

In [2]:
%%cython -a
# distutils: language=c++

from cython cimport boundscheck, wraparound
from libcpp.map cimport map
from libcpp.pair cimport pair
from cython.operator cimport dereference
import numpy as np

ctypedef map[int, int] TOrders
ctypedef pair[int, int] TOrder

ctypedef map[double, TOrders] TBook
ctypedef pair[double, TOrders] TBookItem

cdef TBook Book

cpdef existPrice(double key):
    return Book.find(key) != Book.end()

cpdef Insert(double price, int orderID, int amt):
    """Insert the new order into orderbook"""
    cdef TOrders Orders
    
    if existPrice(price):
        Book[price].insert(TOrder(orderID, amt))
    else:
        Orders.insert(TOrder(orderID, amt))
        Book.insert(TBookItem(price, Orders))
    
cpdef Delete(double price, int orderID):
    """Delete one order from orderbook"""
    if existPrice(price):
        Book[price].erase(orderID)
        if Book[price].size() == 0:
            Book.erase(price)
        
    
def getBook():
    return Book

def firstItem():
    item = dereference(Book.begin()) if not Book.empty() else None
    return item

cdef firstPrice():
    return dereference(Book.begin()).first if not Book.empty() else 0

def lastItem():
    return dereference(Book.rbegin()) if not Book.empty() else None

def lastPrice():
    return dereference(Book.rbegin()).first if not Book.empty() else 0

cdef packed struct orderlog_struct:
    # The struct needs to be packed since by default numpy dtypes aren't aligned
    long OrderID
    double Price
    int Size
    char Action
    char Side

@boundscheck(False)  # Deactivate bounds checking
@wraparound(False)
def getHistory(orderlog_struct[::1] x):
    result = np.zeros((x.shape[0]), dtype=np.double)
    cdef double[::1] result_view = result
    
    cdef double c = 0
    for i in range(x.shape[0]):
        if x[i].Action == 1 and x[i].Price > 0:
            Insert(x[i].Price, x[i].OrderID, x[i].Size)
        c += x[i].OrderID
        result_view[i] = firstPrice()
    return result

In [3]:
X = clickdb.npOrderlog('GAZP', date(2021,12,1))
X

array([(      16, 330.  ,  200, 1, b'B'), (      17, 325.33,  100, 1, b'B'), (      18, 342.  ,  800, 1, b'S'), (      19, 344.94,   40, 1, b'S'),
       (      20, 353.82,  500, 1, b'S'), ..., (19670513, 350.  ,   10, 0, b'S'), (19670932, 348.11,   50, 0, b'B'), (19673480, 347.23,   20, 0, b'B'),
       (19674247, 349.65, 7150, 0, b'S'), (19674894, 349.5 ,  100, 0, b'S')],
      dtype=[('OrderID', '<i8'), ('Price', '<f8'), ('Size', '<i4'), ('Action', 'i1'), ('Side', 'S1')])

In [4]:
%timeit getHistory(X)

getHistory(X)

303 ms ± 4.79 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


array([307.85, 307.85, 307.85, 307.85, 307.85, ..., 307.85, 307.85, 307.85, 307.85, 307.85])

In [5]:
getBook()

{307.85: {30737: 30,
  38523: 100,
  38643: 500,
  38816: 1000,
  44098: 500,
  49675: 70,
  51219: 500,
  59670: 160,
  281005: 20,
  348040: 10,
  356901: 50,
  382316: 3000,
  575036: 40,
  677466: 190,
  762754: 340,
  862705: 30,
  940734: 10000,
  1033255: 10,
  1204368: 3000,
  1318323: 130,
  1975876: 500,
  2059576: 100,
  2902370: 10,
  2980879: 20,
  3255082: 40,
  3255165: 10,
  3801494: 20,
  3890495: 100,
  4066186: 20,
  4260049: 710,
  4260063: 400,
  4617043: 30,
  4674525: 10,
  5009008: 2380,
  5009014: 1560,
  5009016: 1650,
  5009963: 20,
  5231397: 40,
  5338684: 20,
  5338881: 10,
  5648303: 10,
  5958942: 10,
  7377928: 10,
  7428400: 4000,
  8141686: 10,
  8484225: 200,
  8644625: 10,
  8851533: 1000,
  9521362: 10,
  9631296: 50,
  10787400: 10,
  12036646: 20,
  12669485: 20,
  12669506: 90,
  13506423: 2510,
  13506424: 1730,
  13506426: 1730,
  14426154: 20,
  14888558: 10,
  15522372: 40,
  15552415: 40,
  18255742: 20},
 307.86: {13094787: 60},
 307.87: {